APRIL TAG DETECTION, DISTANCE MEASUREMENT AND DIRECTION/POSITION/ORIENTATION

In [ ]:
import cv2
import numpy as np
from pupil_apriltags import Detector


# Start webcam capture
cap = cv2.VideoCapture(0)

ret, frame = cap.read()
if ret:
    frame_height, frame_width = frame.shape[:2]
    frame_center_x = frame_width // 2
    frame_center_y = frame_height // 2
else:
    print("Failed to capture video frame. Exiting...")
    cap.release()
    exit(1)

cap.release()

# Load camera calibration parameters
data = np.load('camera_calibration.npz')
camera_matrix = data['camera_matrix']
distortion_coefficient = data['distortion_coefficient']

# Initialize AprilTag detector
at_detector = Detector(families='tag36h11',
                       nthreads=4,
                       quad_decimate=1.0,
                       quad_sigma=0.0,
                       refine_edges=1,
                       decode_sharpening=0.25,
                       debug=0)

# Start webcam capture
cap = cv2.VideoCapture(0)




while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect AprilTags in the image
    tags = at_detector.detect(gray, estimate_tag_pose=True, camera_params=[camera_matrix[0,0], camera_matrix[1,1], camera_matrix[0,2], camera_matrix[1,2]], tag_size=0.165) # tag_size should be set to the real size of the tag in meters

    for tag in tags:
        # Draw a bounding box around the detected tag
        cv2.polylines(frame, [np.array(tag.corners, dtype=np.int32).reshape((-1,1,2))], True, (0,255,0), 2)

        # Display the tag ID on the image
        cv2.putText(frame, str(tag.tag_id), (int(tag.center[0]), int(tag.center[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Calculate distance from the camera (assuming tvec is in meters)
        distance = np.linalg.norm(tag.pose_t)
        print(f"Distance to tag {tag.tag_id}: {distance} meters")

        # `tag.center[0]` which is the x-coordinate of the center of the tag in the camera frame
        # And `frame_center_x` which is the x-coordinate of the center of the camera frame

        turn_threshold = 50  # How many pixels from the center to consider as a turn
        turning = False

        if distance > 2:
           if abs(tag.center[0] - frame_center_x) > turn_threshold:
              turning = True
              if tag.center[0] > frame_center_x:
                   # The tag (and therefore the person) has moved to the right of the center
                   print(f'TURN RIGHT\n')
              else:
                  # The tag (and therefore the person) has moved to the left of the center
                  print(f'TURN LEFT\n')
           else:
            # The tag is approximately centered, move straight
                 print(f'MOVE FORWARD\n')
        else:
           print(f'STOP\n')

    # Display the resulting frame
    cv2.imshow('Frame', frame)

    # Break the loop with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()




